In [ ]:
pip install XGBoost

In [1]:
## Importing necessary libaries
import boto3
import pandas as pd; pd.set_option('display.max_columns', 50)
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [2]:
## 1. Using the pandas library to read the csv data file and create a data-frames called insurance

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data-448-bucket-callaghan'
bucket = s3.Bucket(bucket_name)

## Defining the csv file
file_key = 'insurance.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

insurance = pd.read_csv(file_content_stream)

insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
## 2. Changing sex, smoker and region from labels to dummy variables.

## Changing labels to numbers
insurance['sex'] = np.where(insurance['sex'] == 'female', 0, 1)
insurance['smoker'] = np.where(insurance['smoker'] == 'no', 0, 1)

## Extracting region dummies
region_dummies = pd.get_dummies(insurance['region']).iloc[:, 0:3]

## Appending dummies
insurance = pd.concat([insurance, region_dummies], axis = 1)

insurance.head()

,age,sex,bmi,children,smoker,region,charges,northeast,northwest,southeast
0,19,0,27.900,0,1,southwest,16884.92400,0,0,0
1,18,1,33.770,1,0,southeast,1725.55230,0,0,1
2,28,1,33.000,3,0,southeast,4449.46200,0,0,1
3,33,1,22.705,0,0,northwest,21984.47061,0,1,0
4,32,1,28.880,0,0,northwest,3866.85520,0,1,0


In [4]:
## 3.  Engineer the interactions/features from Chapter 4 lecture notes (the ones from the decision tree)

## Feature engineering:
insurance['interaction_1'] = np.where((insurance['smoker'] == 0) & (insurance['age'] <= 32.5), 1, 0)

insurance['interaction_2'] = np.where((insurance['smoker'] == 0) & (insurance['age'] > 32.5) & (insurance['age'] <= 44.5), 1, 0)

insurance['interaction_3'] = np.where((insurance['smoker'] == 0) & (insurance['age'] > 44.5) & (insurance['age'] < 51.5), 1, 0)

insurance['interaction_4'] = np.where((insurance['smoker'] == 0) & (insurance['age'] > 51.5), 1, 0)

In [5]:
## 4. Based on the feature selection analysis shown in Chapter 4, it seems that age, bmi, children, smoker, and interaction 4 
## are the top 5 important variables. Using the top variables as input variables and charges as the target variable to split the 
## data into three datasets: train (80%), validation (10%) and test (10%)

## Defining input and target variables
X = insurance[['age','bmi', 'children', 'smoker', 'interaction_4']]
Y = insurance['charges']

## Splitting the data
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2)
X_val, X_test, Y_val, Y_test = train_test_split(X_val, Y_val, test_size = 0.5)

In [6]:
## 5. Using train data-frame and the top 5 features to perform a hyper-tuning job on the random forest model using the 
## GridSearchCV function and the following dictionary:

RF_param_grid = {'n_estimators': [100, 300, 500], 'min_samples_split': [10, 15], 'min_samples_leaf': [5, 7], 'max_depth': [3, 5, 7]}

## Running GridSearchCV with three folds
rf_grid_search = GridSearchCV(RandomForestRegressor(), RF_param_grid, cv = 3, scoring = 'neg_mean_squared_error', 
                             n_jobs = -1).fit(X_train, Y_train)

## Printing the best hyper-parameters
print('Best hyper-parameter combination:', rf_grid_search.best_params_)

## Extracting the optimal model
rf_md = rf_grid_search.best_estimator_

## Using the optimal model to predict the charges on the validation and test set
rf_val_preds = rf_md.predict(X_val)
rf_test_preds = rf_md.predict(X_test)

## Computing the mean squared error of the predictions 
rf_val_mse = mean_squared_error(Y_val, rf_val_preds)
rf_test_mse = mean_squared_error(Y_test, rf_test_preds)

print('\nMean Square Error on the validation set:', round(rf_val_mse))
print('Mean Square Error on the testing set:', round(rf_test_mse))

Best hyper-parameter combination: {'max_depth': 5, 'min_samples_leaf': 7, 'min_samples_split': 15, 'n_estimators': 300}

Mean Square Error on the validation set: 14613715
Mean Square Error on the testing set: 25507001


In [7]:
## 6. Using train data-frame and the top 5 features to perform a hyper-tuning job on the XGBoost model using the 
## GridSearchCV function and the following dictionary:

XGBoost_param_grid = {'n_estimators': [500], 'max_depth': [3, 5, 7], 'min_child_weight': [5, 7], 
                      'learning_rate': [0.01], 'gamma': [0.3, 0.1], 'subsample': [1], 'colsample_bytree': [1]}

## Running GridSearchCV with three folds
xgb_grid_search = GridSearchCV(XGBRegressor(), XGBoost_param_grid, cv = 3, scoring = 'neg_mean_squared_error', 
                             n_jobs = -1).fit(X_train, Y_train)

## Printing the best hyper-parameters
print('Best hyper-parameter combination:', xgb_grid_search.best_params_)

## Extracting the optimal model
xgb_md = xgb_grid_search.best_estimator_

## Using the optimal model to predict the charges on the validation and test set
xgb_val_preds = xgb_md.predict(X_val)
xgb_test_preds = xgb_md.predict(X_test)

## Computing the mean squared error of the predictions 
xgb_val_mse = mean_squared_error(Y_val, xgb_val_preds)
xgb_test_mse = mean_squared_error(Y_test, xgb_test_preds)

print('\nMean Square Error on the validation set:', round(xgb_val_mse))
print('Mean Square Error on the testing set:', round(xgb_test_mse))

Best hyper-parameter combination: {'colsample_bytree': 1, 'gamma': 0.3, 'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 7, 'n_estimators': 500, 'subsample': 1}

Mean Square Error on the validation set: 14076877
Mean Square Error on the testing set: 26615155


In [ ]:
## 7. Based on the results from parts 5 and 6, we would use the Random Forest model to predict charges 
## because of the lower mean-square error value on the testing set.